Crawl and preprocess tweets of given users

#Imports

In [ ]:
!pip install twitter
!pip install tweepy
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 KB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tweepy
import re
from twitter import *
import requests
import json
import csv
import jsonlines
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle

#Setup

In [ ]:
# API keys 
api_key = ""
api_secrets = ""
bearer_token = ""
access_token = ""
access_secret = ""

# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
 
api = tweepy.API(auth)
# api = tweepy.API(auth, wait_on_rate_limit=True, retry_count=10, retry_delay=5, retry_errors=set([503]))
 
try:
    api.verify_credentials()
    print('Successful Authentication')
except:
    print('Failed authentication')

Successful Authentication


In [ ]:
path = "/content/drive/My Drive/tweets-dataset/"

In [ ]:
# df = pd.read_csv(path+'gs_scholars_candidate_twitter_accounts.csv', index_col=0, nrows=501)
# df['google_scholar_id'] = df['url'].apply(lambda x : x.split("user=")[1])
# df['twitter_id'] = df['url of their twitter'].apply(lambda x : x.split(".com/")[-1] if x==x else x)
# df[['name', 'google_scholar_id', 'twitter_id']].to_csv(path+"gs_id2twitter_id.csv")
# df = df[['name', 'google_scholar_id', 'twitter_id']]
# df.head()

In [ ]:
gs_link_prefix = "https://scholar.google.com/citations?user=" 
twitter_link_prefix = "https://twitter.com/"

#Crawl Tweets

##### Prev fns

In [ ]:
# def get_tweets_simple(twitter_name, api):
#     try:
#         tweets = api.user_timeline(screen_name=twitter_name, count=10, tweet_mode="extended")
#         tweets_list = []
#         for tweet in tweets:
#             json_object = tweet._json
#             tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
#                           "text": json_object["full_text"]}
#             # get full text from retweets
#             if "retweeted_status" in json_object:
#                 tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
#                               "text": json_object["retweeted_status"]["full_text"]}
#             # change the form of link in the tweets
#             urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', tweet_info["text"])
#             for each in urls:
#                 try:
#                     r = requests.get(each)
#                     tweet_info["text"] = tweet_info["text"].replace(each, r.url)
#                 except:
#                     continue
#             tweets_list.append(tweet_info)
#         return tweets_list
#     except:
#         return None


# def get_tweets_simple_qnqq(twitter_name, api):
#     try:
#         tweets = api.user_timeline(screen_name=twitter_name, count=10, tweet_mode="extended")
#         tweets_list = []
#         for tweet in tweets:
#             json_object = tweet._json
#             tweets_list.append(json_object)
#         return tweets_list
#     except:
#         return None

# def get_tweets_qnqq(twitter_name, api):
#     try:
#         tweets = api.user_timeline(screen_name=twitter_name, count=100, tweet_mode="extended")
#         tweets_list = []
#         for tweet in tweets:
#             json_object = tweet._json
#             tweets_list.append(json_object)
#         return tweets_list
#     except:
#         return None

# def get_tweets(twitter_name, api):
#     current_time = time.time()
#     try:
#         tweets = api.user_timeline(screen_name=twitter_name, count=5, tweet_mode="extended")
#         print(f"time crawing one tweets is {time.time()-current_time}")
#         current_time = time.time()
#         tweets_list = []
#         for tweet in tweets:
#             json_object = tweet._json
#             tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
#                           "text": json_object["full_text"]}
#             # get full text from retweets
#             if "retweeted_status" in json_object:
#                 tweet_info = {"tweet_id": json_object["id"], "num_of_likes": json_object["favorite_count"],
#                               "text": json_object["retweeted_status"]["full_text"]}
#             # change the form of link in the tweets
#             urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', tweet_info["text"])
#             for each in urls:
#                 try:
#                     r = requests.get(each)
#                     tweet_info["text"] = tweet_info["text"].replace(each, r.url)
#                 except:
#                     continue
#             tweets_list.append(tweet_info)
#         print(f"time processing one tweets is {time.time()-current_time}")
#         return tweets_list
#     except:
#         return None

# def get_full_tweets(twitter_name, api):
#     pass

##### New fns


Columns in dataset
- created_at
- id
- full_text
- user
- place
- quote_count
- reply_count
- retweet_count
- favorite_count - number of likes
- entities
- possibly_sensitive
- lang
- in_reply_to_status_id_str
- in_reply_to_user_id_str
- in_reply_to_screen_name
- is_quote_status
- quoted_status_id_str
- is_retweet

In [ ]:
def full_text_extractor(tweet):
    # if there is 'extended_tweet', return full_text
    if tweet.__contains__('extended_tweet'):
        return tweet['extended_tweet']['full_text']
    # if there is 'retweeted_status' and extended_tweet in it, 
    # return full_text. Otherwise, return the text.
    elif tweet.__contains__('retweeted_status'):
        if tweet['retweeted_status'].__contains__('extended_tweet'):
            return tweet['retweeted_status']['extended_tweet']['full_text']
        else:
            return tweet['text']
    else:
        return tweet['text']

def get_rt_full_text(json_object):
  return json_object['full_text'][:json_object['full_text'].find(":")+2]+json_object['retweeted_status']['full_text']

def get_user_info(user_json):
  user_fields = ["id", "name", "screen_name", "location", "followers_count",\
                 "friends_count", "created_at", "favourites_count", "verified",\
                 "statuses_count", "lang", "profile_image_url"]
  user_info = {}
  for field in user_fields:
    user_info[field] = user_json[field]
  return user_info

def get_tweets(twitter_name, api, num_tweets):
  try:
    tweets = api.user_timeline(screen_name=twitter_name, count=num_tweets, tweet_mode="extended")
    tweets_list = []
    for i, tweet in enumerate(tweets):
      json_object = tweet._json

      tweet_info = {"created_at": json_object["created_at"],\
                    "tweet_id" : json_object["id"], \
                    # "text": get_full_text(json_object),\
                    "text" : json_object["full_text"], \
                    "like_count" : json_object["favorite_count"], \
                    "quote_count" : 0,\
                    "reply_count" : 0,\
                    "retweet_count" : json_object["retweet_count"],\
                    "user" : get_user_info(json_object["user"]),\
                    "place": json_object["place"],\
                    "lang" : json_object["lang"],\
                    "entities" : json_object["entities"],\
                    "possibly_sensitive" : False,\
                    "is_quote_status" : False,\
                    "in_reply_to_user_id" : json_object["in_reply_to_user_id"],\
                    "in_reply_to_screen_name" : json_object["in_reply_to_screen_name"],\
                    "is_retweet" : False
                    }
      if "quoted_status_id" in json_object:
        tweet_info["is_quote_status"] = True
      if "quote_count" in json_object:
        tweet_info["quote_count"] =  json_object["quote_count"]
      if "reply_count" in json_object:
        tweet_info["reply_count"] = json_object["reply_count"]
      if "possibly_sensitive" in json_object:
        tweet_info["possibly_sensitive"] = json_object["possibly_sensitive"]
      if "retweeted_status" in json_object:
        tweet_info["is_retweet"] = True
        tweet_info["text"] = get_rt_full_text(json_object)
      tweets_list.append(tweet_info)
    return tweets_list
  except:
    return None

In [ ]:
users = api.search_users("Zhijing Jin")
users

In [ ]:
api.get_user("ZhijingJin")._json

In [ ]:
for user in users:
  print(user._json['screen_name'])
  tweets = get_tweets(user._json['screen_name'], api, 10)
  for tweet in tweets:
    print(tweet['text'])
  print("\n")

In [ ]:
users[2]._json

In [ ]:
total_tweets = 284176
save_dataset = True

In [ ]:
with open(path+"/gs_scholars_candidate_twitter_accounts_78k.tsv","r") as gs_f_r:
  reader = csv.reader(gs_f_r, delimiter="\t")
  for i, line in enumerate(reader):
    index = line[0]
    if i==0:
      continue
    name = line[1]
    candidate_ids = [id for id in line[4:] if len(id)>0]
    for rank, c_id in enumerate(candidate_ids):
        tweets = get_tweets(c_id, api, 1000)
        if tweets is None:
          print(index, rank, name, c_id)
          continue
        total_tweets += len(tweets)
        if save_dataset:
          with open(path + 'final-dataset-78K/' + str(index) + '_' + str(rank) + '_' + c_id +'.pkl', 'wb') as gs_f_w:
            pickle.dump(tweets, gs_f_w)
    # time.sleep(1)
    if int(index)!=0 and int(index)%50 == 0:
      print(f"Tweets crawled for .. {index}")

In [ ]:
with open(path+"/gs_scholars_candidate_twitter_accounts.tsv","r") as f_r:
  reader = csv.reader(f_r, delimiter="\t")
  for i, line in enumerate(reader):
    index = line[0]
    if i==0:
      continue
    name = line[1]
    twitter_url = line[4]
    candidate_ids = [id for id in line[6:] if len(id)>0]
    for rank, c_id in enumerate(candidate_ids):
      # print(index, rank, name, c_id)
        tweets = get_tweets(c_id, api, 1000)
        if tweets is None:
          print(index, rank, name, c_id)
          continue
        total_tweets += len(tweets)
        if save_dataset:
          with open(path + 'final-dataset/' + str(index) + '_' + str(rank) + '_' + c_id +'.pkl', 'wb') as f_w:
            pickle.dump(tweets, f_w)
    time.sleep(1)
    if int(index)!=0 and int(index)%50 == 0:
      print(f"Tweets crawled for .. {index}")

In [ ]:
total_tweets

284176

Number of tweets to crawl per author

In [ ]:
max_tweets_per_author = 1000
df['twitter_id'].notnull().sum(), df['twitter_id'].notnull().sum()*max_tweets_per_author

(173, 173000)

In [ ]:
total_tweets = 0
for i, row in df.iterrows():
  if row['twitter_id']!=row['twitter_id']:
    continue
  tweets = get_tweets(row['twitter_id'], api, max_tweets_per_author)
  if tweets is None:
    print(print(i, row['twitter_id']))
    continue
  total_tweets += len(tweets)
  if save_dataset:
    with open(path + 'dataset/' + str(i) + '_' + row['twitter_id'] +'.pkl', 'wb') as f:
      pickle.dump(tweets, f)
  # time.sleep(1)
  if i!=0 and i%50 == 0:
    print("Tweets crawled for .. " + str(i))  

In [ ]:
total_tweets

16025

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
tweets_dataset = []
onlyfiles = [f for f in listdir(path+'dataset/') if isfile(join(path+'/dataset/', f))]
for filename in onlyfiles:
  tweets_dataset.extend(np.load(path+'/dataset/'+filename, allow_pickle=True))
print(len(tweets_dataset))
with open(path + 'tweets_500.pkl', 'wb') as f:
    pickle.dump(tweets_dataset, f)

16025


In [ ]:
num_retweets = 0
for tweet in tweets_dataset:
  if tweet['text'][:2] == "RT":
    num_retweets += 1

In [ ]:
num_retweets

6806

In [ ]:
usuable_tweets = total_tweets - num_retweets
usuable_tweets

9219

In [ ]:
class_0, class_1 = 0, 0
popular_names = set()
for tweet in tweets_dataset:
  if tweet['text'][:2] != "RT":
    if tweet['like_count'] >= 100:
      popular_names.add(tweet['user']['screen_name'])
      class_1 += 1
    else:
      class_0 += 1 

In [ ]:
class_0, class_1

(9088, 131)

In [ ]:
(class_1/usuable_tweets)*100, (class_0/usuable_tweets)*100

(1.4209784141447013, 98.5790215858553)